In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install transformers rouge-score nltk datasets rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.3 MB/s 
     |████████████████████████████████| 451 kB 57.3 MB/s 
     |████████████████████████████████| 182 kB 53.8 MB/s 
     |████████████████████████████████| 7.6 MB 53.4 MB/s 
     |████████████████████████████████| 115 kB 43.6 MB/s 
     |████████████████████████████████| 212 kB 32.0 MB/s 
     |████████████████████████████████| 127 kB 64.0 MB/s 
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=0bc5f80c6ef1853bee0a5ff50e6796cf0595186e9307cc6103afc413514d7388
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
import re
import nltk
import transformers
from datasets import load_dataset, load_metric
import string
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
model_checkpoint = "t5-base"
TOKENIZER = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [6]:
path = "/content/gdrive/My Drive/cse_842_project/"

In [7]:
data_files = {"train": path + "bbc_data/train_split_2.csv", "test": path + "bbc_data/test_split_2.csv"}
dataset = load_dataset("csv", data_files=data_files)

# DatasetDict({
#    train: Dataset({
#        features: ['filepath', 'class', 'title', 'content',],
#        num_rows: 192368
#    })
# })
# DatasetDict({
#    train: Dataset({
#        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags'],
#        num_rows: 192368
#    })
# })

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f8f2c73a88edc461/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# getting dataset counts
print('Total train data is : {}'.format(len(dataset['train'])))
print('Total test data is : {}'.format(len(dataset['test'])))

Total train data is : 1977
Total test data is : 349


In [12]:
def clean_string(s):
    # add regex to remove urls and bractetted stuff.
    s = re.sub(r'^https?:\/\/.*[\r\n]*', '', s, flags=re.MULTILINE)
    s = re.sub('\s+', ' ', s)
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub('\s+', ' ', s)
    return s

In [13]:
# selecting only the first paragraph
prefix = "summarize: "

max_input_length = 512
max_target_length = 64

def clean_text(text):
    text = text.split('\n\n')[0]
    sentences = nltk.sent_tokenize(text.strip())
    sentences_cleaned = [clean_string(s) for sent in sentences for s in sent.split('\n')]
    sentences_cleaned_no_titles = [
        sent for sent in sentences_cleaned
        if len(sent) > 0
    ]
    text_cleaned = " ".join(sentences_cleaned_no_titles)
    return text_cleaned

def preprocess_data(examples, tokenizer=TOKENIZER):
    texts_cleaned = [clean_text(text) for text in examples["content"]]
    inputs = [prefix + text for text in texts_cleaned]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    title_cleaned = [clean_text(text) for text in examples["title"]]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            title_cleaned, max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
# datasets_cleaned = dataset.filter(lambda example: (len(example['text']) >= 500) and (len(example['title']) >= 20))
tokenized_dataset = dataset.map(preprocess_data, batched=True)
tokenized_dataset

  0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['file_path', 'class', 'title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1977
    })
    test: Dataset({
        features: ['file_path', 'class', 'title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 349
    })
})

## Finetuning T5 

In [ ]:
# !rm -r {model_dir}

rm: cannot remove '/content/gdrive/My': No such file or directory
rm: cannot remove 'Drive/cse_842_project/models/t5/t5-base-title-generation/checkpoint-400': No such file or directory


In [ ]:
batch_size = 4
model_name = "t5-base-title-generation"
model_dir = path + 'models/t5_single_para/{}'.format(model_name)
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    # report_to="tensorboard"
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(TOKENIZER)

In [16]:
import numpy as np

metric = load_metric("rouge")

def compute_metrics(eval_pred, tokenizer=TOKENIZER):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=TOKENIZER,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/pytorch_model.bin
All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.
Using cuda_amp half precision backend


In [ ]:
trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,2.995500,2.433646,34.351700,8.341200,31.687700,31.682100,10.163300
200,2.544700,2.336045,34.875000,9.056700,32.498900,32.474200,9.693400
300,2.561000,2.277128,35.123700,9.714000,32.948400,33.037400,9.991400
400,2.383200,2.232541,35.653800,9.450400,33.217100,33.214700,9.816600
500,2.348000,2.206903,35.297600,9.419900,33.200000,33.178000,9.796600
600,2.130700,2.173308,35.862600,9.738000,33.045900,33.001200,9.939800
700,2.171600,2.174603,35.539800,9.709400,33.150400,33.167500,9.948400
800,2.106200,2.152591,35.304200,9.995200,33.018100,33.015100,10.065900
900,2.036500,2.140666,36.236800,10.319200,33.733600,33.752600,9.873900
1000,2.088000,2.126839,36.195300,10.618700,33.659800,33.708200,9.853900


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: class, file_path, title, content. If class, file_path, title, content are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 349
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: class, file_path, title, content. If class, file_path, title, content are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 349
  Batch size = 4
Saving model checkpoint to /content/gdrive/My Drive/cse_842_project/models/t5_single_para/t5-base-title-generation/checkpoint-200
Configuration saved in /content/gdrive/My Drive/cse_842_project/models/t5_single_para/t5-base-title-generation/check

TrainOutput(global_step=1980, training_loss=2.0970067881574534, metrics={'train_runtime': 1064.1306, 'train_samples_per_second': 7.431, 'train_steps_per_second': 1.861, 'total_flos': 927018261918720.0, 'train_loss': 2.0970067881574534, 'epoch': 4.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation
Configuration saved in /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/config.json
Model weights saved in /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/pytorch_model.bin
tokenizer config file saved in /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/tokenizer_config.json
Special tokens file saved in /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/special_tokens_map.json


In [17]:
model_name = "t5-base-title-generation/checkpoint-1800"
model_dir = path + 'models/t5_single_para/{}'.format(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

In [18]:
import torch

# get test split
test_tokenized_dataset = tokenized_dataset["test"]

# # pad texts to the same length
def preprocess_test(examples):
    texts_cleaned = [clean_text(text) for text in examples['content']]
    inputs = [prefix + text for text in texts_cleaned]
    # inputs = [prefix + text for text in examples["content"]]
    model_inputs = tokenizer(
        inputs, max_length=max_input_length, truncation=True, padding="max_length"
    )
    return model_inputs

test_tokenized_dataset = test_tokenized_dataset.map(preprocess_test, batched=True)

# prepare dataloader
test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=4)
print('dataloader prepared ....')

# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
    print(i)
    predictions = model.generate(**batch)
    all_predictions.append(predictions)

print('got predictions ....')

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
title_cleaned = [clean_text(text) for text in test_tokenized_dataset["title"]]

all_titles = tokenizer(
    title_cleaned, max_length=max_target_length,
    truncation=True, padding="max_length"
)["input_ids"]

# compute metrics
predictions_labels = [all_predictions_flattened, all_titles]
compute_metrics(predictions_labels)

  0%|          | 0/1 [00:00<?, ?ba/s]

dataloader prepared ....
0


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
got predictions ....


{'rouge1': 36.2622,
 'rouge2': 10.6347,
 'rougeL': 33.5301,
 'rougeLsum': 33.6828,
 'gen_len': 9.8883}

In [19]:
compute_metrics(predictions_labels)

{'rouge1': 36.2622,
 'rouge2': 10.6347,
 'rougeL': 33.5301,
 'rougeLsum': 33.6828,
 'gen_len': 9.8883}

In [20]:
def convert_to_string(input_id):
    tokens = tokenizer.convert_ids_to_tokens(input_id)
    sentence = ''
    for tok in tokens:
        if tok == '</s>':
            break
        elif tok != '<pad>':
            sentence += tok
        else:
            continue
    
    sentence = sentence.replace('▁', ' ').strip()
    return sentence 


In [21]:
convert_to_string(all_titles[0])

'yukos unit buyer faces loan claim'

In [22]:
true_titles = []
for title in all_titles:
    true_titles.append(convert_to_string(title))


predicted_titles = []
for pre_title in all_predictions_flattened:
    predicted_titles.append(convert_to_string(pre_title))




In [23]:
from rouge import Rouge
rouge = Rouge()

scores = rouge.get_scores(predicted_titles, true_titles, avg=True)

scores

{'rouge-1': {'r': 0.3337767771865199,
  'p': 0.33722447561989144,
  'f': 0.330882421203095},
 'rouge-2': {'r': 0.09890162368672395,
  'p': 0.10099604311638695,
  'f': 0.09841724500283626},
 'rouge-l': {'r': 0.31665302224041536,
  'p': 0.3208238752651365,
  'f': 0.3142816548888618}}

In [ ]:
for t,p in zip(true_titles, predicted_titles):
    print(t)
    print(p)
    print('\n\n')

election could be terror target
terror threat real blair warns



rangers seal old firm win
rangers celtic



s korean lender faces liquidation
lg card to be liquidated if it fails to back bail



hacker threat to apple s itunes
apple s itunes users face security threat



guantanamo man suing government
terror suspect to sue guantanamo



qantas considers offshore option
qantas could move jobs out of australia



tarantino to make friday sequel
tarantino in talks over new film



asia shares defy post quake gloom
tsunamis hit thailand and sri lanka



s korea spending boost to economy
south korea to boost economy



lewsey puzzle over disallowed try
lewsey denied late try



potter director signs warner deal
cuaron signs hollywood deal



air passengers win new eu rights
eu rules to increase compensation



fox attacks blair s tory lies
blair no qualms about lying in election



nhs waiting time target is cut
wales hospital waiting targets drop



henman hopes ended in dubai
henman sl